In [1]:
# These are probably more imports than we need. Copied from comparison tool
import pandas as pd
import os
import sys
import time
import shapely
import warnings
from pandarallel import pandarallel

# This module is an easy wrapper to parallelize Panda's apply method.
pandarallel.initialize(
    progress_bar=True,
    # If nb_workers is not set, it defaults to available cores.
    nb_workers=8,
)


module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

from data_pipeline.config import settings
from data_pipeline.etl.base import ExtractTransformLoad
from data_pipeline.etl.sources.census.etl import CensusETL
from data_pipeline.utils import unzip_file_from_url

# If field names are necessary, import them.
# from data_pipeline.score import field_names

# Turn on TQDM for pandas so that we can have progress bars when running `apply`.
from tqdm.notebook import tqdm_notebook

tqdm_notebook.pandas()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [20]:
# Define some input fields
LAT_FIELD = "Latitude"
LONG_FIELD = "Longitude"
KEY_FIELD = "AMLIS Key"


# TODO: switch to whole US
GEOJSON_PATH = CensusETL().GEOJSON_PATH / "us.json"
# GEOJSON_PATH = CensusETL().GEOJSON_PATH / "02.json"
GEOJSON_TRACT_ID_FIELD = "GEOID10"

# Choose output directory:
OUTPUT_DIR = ExtractTransformLoad.DATA_PATH / "abandoned_mine_lands"
# Create directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

2022-07-11 19:42:16,399 [data_pipeline.etl.sources.census.etl_utils] INFO     Downloading fips from S3 repository
2022-07-11 19:42:16,403 [data_pipeline.utils] INFO     Downloading https://justice40-data.s3.amazonaws.com/data-sources/fips_states_2010.zip
2022-07-11 19:42:16,801 [data_pipeline.utils] INFO     Extracting /Users/lucas/Documents/usds/repos/justice40-tool/data/data-pipeline/data_pipeline/data/tmp/downloaded-b6413152-455e-47a5-817a-a96e3a892044.zip


In [3]:
# Load geojson
import geopandas

t1 = time.time()
census_tract_gdf = geopandas.read_file(
    GEOJSON_PATH,
    # Use `pyogrio` because it's vectorized and faster.
    engine="pyogrio",
)

t2 = time.time()

print(f"Code took {str(t2-t1)} seconds.")

print(census_tract_gdf)

Code took 884.5575151443481 seconds.
      STATEFP10 COUNTYFP10 TRACTCE10      GEOID10  NAME10  \
0            27        139    080202  27139080202  802.02   
1            27        139    080204  27139080204  802.04   
2            27        139    080100  27139080100     801   
3            27        139    080302  27139080302  803.02   
4            27        139    080400  27139080400     804   
...         ...        ...       ...          ...     ...   
74129        16        005    001601  16005001601   16.01   
74130        16        005    001300  16005001300      13   
74131        16        005    001000  16005001000      10   
74132        16        005    000900  16005000900       9   
74133        16        005    000800  16005000800       8   

                NAMELSAD10 MTFCC10 FUNCSTAT10   ALAND10  AWATER10  \
0      Census Tract 802.02   G5020          S   5137595    109563   
1      Census Tract 802.04   G5020          S   4730968    120879   
2         Census Tract 

In [4]:
# Create temporary path
tmp_path = ExtractTransformLoad.DATA_PATH / "tmp" / "abandoned_mine_lands"
# Create directory if it doesn't exist
tmp_path.mkdir(parents=True, exist_ok=True)

eamlis_path_in_s3 = (
    settings.AWS_JUSTICE40_DATASOURCES_URL + "/eAMLIS export of all data.tsv.zip"
)

unzip_file_from_url(
    file_url=eamlis_path_in_s3,
    download_path=tmp_path,
    unzipped_file_path=tmp_path,
)

eamlis_path = tmp_path + "/eAMLIS export of all data.tsv"

eamlis_source_df = pd.read_csv(
    filepath_or_buffer=eamlis_path,
    sep="\t",
)

eamlis_source_df.head()

/var/folders/lx/xmq8p65j71v9xq2bhsd2j5w40000gp/T/ipykernel_21791/630537932.py:6: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  eamlis_source_df = pd.read_csv(


,AMLIS Key,State/Tribe,County,Congressional District,Quadrangle Name,Watershed,HUC Code,FIPS Code,Latitude,Longitude,...,Unfunded Metric Units,Funded Standard Units,Funded Costs,Funded GPRA Acres,Funded Metric Units,Completed Standard Units,Completed Costs,Completed GPRA Acres,Completed Metric Units,Unnamed: 40
0,AK000001,AK,MATANUSKA-SUSITNA,1.0,ANCHORAGE C-8,NaN,NaN,02170,61.6,-149.8,...,0.0,0.0,0.0,0.0,0.0,2.0,10000.0,0.20,2.0,NaN
1,AK000001,AK,MATANUSKA-SUSITNA,1.0,ANCHORAGE C-8,NaN,NaN,02170,61.6,-149.8,...,0.0,0.0,0.0,0.0,0.0,4.0,20000.0,0.40,4.0,NaN
2,AK000001,AK,MATANUSKA-SUSITNA,1.0,ANCHORAGE C-8,NaN,NaN,02170,61.6,-149.8,...,0.0,0.0,0.0,0.0,0.0,900.0,33200.0,12.86,274.3,NaN
3,AK000002,AK,FAIRBANKS NORTH STAR,1.0,Fairbanks D-3,19030004,NaN,02090,64.8,-148.0,...,0.0,0.0,0.0,0.0,0.0,8.0,35324.0,0.80,8.0,NaN
4,AK000002,AK,FAIRBANKS NORTH STAR,1.0,Fairbanks D-3,19030004,NaN,02090,64.8,-148.0,...,0.0,0.0,0.0,0.0,0.0,1.0,4416.0,0.10,1.0,NaN


In [5]:
mines_df = eamlis_source_df

print(mines_df.columns)

# TODO: investigate how to combine multiple rows for the same lat/long.
# This just keeps one of the rows arbitrarily. We might need additional columns of information.
mines_unique_df = mines_df.drop_duplicates(subset=[LAT_FIELD, LONG_FIELD], keep="last")

# TODO: investigate whether other columns (such as mine problem severity) are needed.
mines_unique_df = mines_unique_df[[KEY_FIELD, LAT_FIELD, LONG_FIELD]]

mines_unique_df.head()

Index(['AMLIS Key', 'State/Tribe', 'County', 'Congressional District',
       'Quadrangle Name', 'Watershed', 'HUC Code', 'FIPS Code', 'Latitude',
       'Longitude', 'Funding Source / Program', 'Problem Area Name',
       'Problem Area Number', 'Planning Unit Name', 'Planning Unit Number',
       'Problem Priority', 'Problem Type', 'Mining Type', 'Ore Types',
       'Date Prepared', 'Date Revised', 'Private Owner %', 'State Owner %',
       'Other Federal Owner %', 'Park Service Owner %',
       'Forest Service Owner %', 'Indian Owner %', 'BLM Owner %',
       'Unfunded Standard Units', 'Unfunded Costs', 'Unfunded GPRA Acres',
       'Unfunded Metric Units', 'Funded Standard Units', 'Funded Costs',
       'Funded GPRA Acres', 'Funded Metric Units', 'Completed Standard Units',
       'Completed Costs', 'Completed GPRA Acres', 'Completed Metric Units',
       'Unnamed: 40'],
      dtype='object')


,AMLIS Key,Latitude,Longitude
2,AK000001,61.6,-149.8
6,AK000003,61.6,-144.0
12,AK000006,61.7,-149.0
25,AK000012,61.6,-148.9
30,AK000015,61.7,-148.2


In [7]:
# METHOD DEFINITIONS
def get_census_tract_for_one_coordinate(
    geom_point: shapely.geometry.point.Point,
    census_tract_gdf: geopandas.geodataframe.GeoDataFrame,
) -> str:
    # GEOJSON_TRACT_ID_FIELD

    # geopandas' contain method works row to row.
    # So create a duplicate row for the point across the length of the census tract gdf
    #     number_of_census_tracts = len(census_tract_gdf)
    #     point_as_gdf = geopandas.GeoDataFrame([[geom_point] * number_of_census_tracts])

    # Now run a row-to-row contains
    #     print(point_as_gdf)

    contains_result = census_tract_gdf.contains(geom_point)
    count_of_census_tract_matches = len(census_tract_gdf[contains_result])

    if count_of_census_tract_matches == 0:
        warnings.warn(
            f"Warning: no tract matches for {geom_point}",
            DeprecationWarning,
            stacklevel=2,
        )
        census_tract_id = None

    elif count_of_census_tract_matches > 1:
        warnings.warn(
            f"Warning: too many tract matches for {geom_point}",
            DeprecationWarning,
            stacklevel=2,
        )
        census_tract_id = None

    else:
        # With only one tract returned, extract the ID.
        census_tract_id = census_tract_gdf[contains_result][
            GEOJSON_TRACT_ID_FIELD
        ].values[0]

    return census_tract_id


def get_census_tracts_for_geom_points(
    points_gdf: geopandas.geodataframe.GeoDataFrame,
    census_tract_gdf: geopandas.geodataframe.GeoDataFrame,
) -> geopandas.geodataframe.GeoDataFrame:
    geometry_column_name = "geometry"
    result_gdf = points_gdf.parallel_apply(
        lambda frame: get_census_tract_for_one_coordinate(
            geom_point=frame[geometry_column_name], census_tract_gdf=census_tract_gdf
        ),
        axis=1,
    )
    return result_gdf


def get_census_tracts_for_dataframe_with_lat_long(
    coordinates_df: pd.DataFrame,
    latitude_column: str = LAT_FIELD,
    longitude_column: str = LONG_FIELD,
    census_tract_gdf: geopandas.geodataframe.GeoDataFrame = census_tract_gdf,
):
    # Avoid these side-effects by creating a duplicate.
    coordinates_df_duplicate = coordinates_df

    # First, convert the plain DataFrame into a geopandas data frame with lat/long geometry points.
    coordinates_geopandas_gdf = geopandas.GeoDataFrame(
        coordinates_df_duplicate,
        geometry=geopandas.points_from_xy(
            x=coordinates_df_duplicate[longitude_column],
            y=coordinates_df_duplicate[latitude_column],
        ),
    )

    # Find the tract IDs for each point.
    tract_results = get_census_tracts_for_geom_points(
        points_gdf=coordinates_geopandas_gdf, census_tract_gdf=census_tract_gdf
    )

    # Join the tract IDs back on the original dataframe
    coordinates_with_tracts_df = coordinates_df
    coordinates_with_tracts_df[
        ExtractTransformLoad.GEOID_TRACT_FIELD_NAME
    ] = tract_results

    # Remove unnecessary `geometry` column
    # For unclear reasons, the initial `GeoDataFrame` creates a `geometry` column on the input dataframe that we don't want.
    coordinates_with_tracts_df = coordinates_with_tracts_df.drop("geometry", axis=1)

    return coordinates_with_tracts_df

In [8]:
t1 = time.time()

# Takes ~26 minutes with 4,000 rows.
mines_unique_with_tracts_df = get_census_tracts_for_dataframe_with_lat_long(
    coordinates_df=mines_unique_df
)

t2 = time.time()

print(f"Code took {str(t2-t1)} seconds.")

print(mines_unique_with_tracts_df)

      AMLIS Key  Latitude  Longitude
2      AK000001      61.6     -149.8
6      AK000003      61.6     -144.0
12     AK000006      61.7     -149.0
25     AK000012      61.6     -148.9
30     AK000015      61.7     -148.2
...         ...       ...        ...
57140  WY216747      42.9     -108.1
57145  WY242429      41.8     -106.8
57146  WY242431      42.5     -108.7
57147  WY242441      42.8     -107.4
57148  WY242444      42.6     -110.9

[3977 rows x 3 columns]


/Users/lucas/.virtualenvs/scoring2/lib/python3.9/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


/var/folders/lx/xmq8p65j71v9xq2bhsd2j5w40000gp/T/ipykernel_21791/3033776402.py:55: DeprecationWarning: Warning: no tract matches for POINT (-130 55.9)
  lambda frame: get_census_tract_for_one_coordinate(


Code took 1550.5722029209137 seconds.
      AMLIS Key  Latitude  Longitude GEOID10_TRACT
2      AK000001      61.6     -149.8   02170000401
6      AK000003      61.6     -144.0   02261000100
12     AK000006      61.7     -149.0   02170000200
25     AK000012      61.6     -148.9   02170001300
30     AK000015      61.7     -148.2   02170000200
...         ...       ...        ...           ...
57140  WY216747      42.9     -108.1   56013000300
57145  WY242429      41.8     -106.8   56007968100
57146  WY242431      42.5     -108.7   56013000300
57147  WY242441      42.8     -107.4   56025001800
57148  WY242444      42.6     -110.9   56023978100

[3977 rows x 4 columns]


In [22]:
mines_unique_with_tracts_df.to_csv(OUTPUT_DIR / "abandoned_mine_lands.csv", index=False)

In [27]:
len(mines_unique_with_tracts_df[ExtractTransformLoad.GEOID_TRACT_FIELD_NAME].unique())

2035